# Build a Bioinformatics Solubility Dashboard in Snowflake

In this notebook, you'll build a **bioinformatics project** from scratch in Snowflake. 

Briefly, we're using the *Delaney* solubility data set. Solubility is an important property for successful drug discovery efforts and is amongst one of the key metrics used in defining drug-like molecules according to the Lipinski Rule of 5.

In a nutshell, here's what you're building:
- Load data into Snowflake
- Perform data preparation using Pandas
- Build a simple dashboard with Streamlit


## About molecular solubility

Molecular solubility is a crucial property in drug development that affects whether a drug can reach its target in the human body. Let me explain why it matters in simple terms.

### Solubility
Solubility is a molecule's ability to dissolve in a liquid, which literally means the ability to dissolve in human bloodstream and transport to its desired target in the human body. If it can dissolve, it can't work!

Poorly soluble drugs might require higher doses or special formulations, leading to potential side effects or complicated treatment regimens. So we want drugs that are both effective and yet soluble so that fewer of it is required so as to minimize potential side effects.

### Lipinski's Rule of 5
Drug development often refer to a guidelines known as the Lipinski's Rule of 5 to predict whether a molecule will be soluble enough to make a good oral drug. This includes factors like:
- Molecule's size
- How water-loving or water-repelling it is
- Number of hydrogen bond donors and acceptors

Understanding and optimizing solubility helps pharmaceutical companies develop effective medicines that can be easily administered and work efficiently in the body.

## Load data

Here, we're loading the Delaney data set ([reference](https://pubs.acs.org/doi/10.1021/ci034243x)).

In [ ]:
SELECT * FROM CHANINN_DEMO_DATA.PUBLIC.SOLUBILITY

## Convert SQL output to Pandas DataFrame

We're using `to_pandas()` method to convert our SQL output table to a Pandas DataFrame.

In [ ]:
sql_data.to_pandas()

## Data Aggregation

Here, we're aggregating the data (grouping it) by its molecular weight:
- `small` if <300
- `large` if >= 300

In [ ]:
df['MOLWT_CLASS'] = pd.Series(['small' if x < 300 else 'large' for x in df['MOLWT']])
df_class = df.groupby('MOLWT_CLASS').mean().reset_index()
df_class

## Building the Solubility Dashboard

In [ ]:
import streamlit as st

st.title('☘️ Solubility Dashboard')

# Data Filtering
mol_size = st.slider('Select a value', 100, 500, 300)
df['MOLWT_CLASS'] = pd.Series(['small' if x < mol_size else 'large' for x in df['MOLWT']])
df_class = df.groupby('MOLWT_CLASS').mean().reset_index()

st.divider()

# Calculate Metrics
molwt_large = round(df_class['MOLWT'][0], 2)
molwt_small = round(df_class['MOLWT'][1], 2)
numrotatablebonds_large = round(df_class['NUMROTATABLEBONDS'][0], 2)
numrotatablebonds_small = round(df_class['NUMROTATABLEBONDS'][1], 2)
mollogp_large = round(df_class['MOLLOGP'][0], 2)
mollogp_small = round(df_class['MOLLOGP'][1], 2)
aromaticproportion_large = round(df_class['AROMATICPROPORTION'][0], 2)
aromaticproportion_small = round(df_class['AROMATICPROPORTION'][1], 2)

# Data metrics and visualizations
col = st.columns(2)
with col[0]:
    st.subheader('Molecular Weight')
    st.metric('Large', molwt_large)
    st.metric('Small', molwt_small)
    st.bar_chart(df_class, x='MOLWT_CLASS', y='MOLWT', color='MOLWT_CLASS')

    st.subheader('Number of Rotatable Bonds')
    st.metric('Large', numrotatablebonds_large)
    st.metric('Small', numrotatablebonds_small)
    st.bar_chart(df_class, x='MOLWT_CLASS', y='NUMROTATABLEBONDS', color='MOLWT_CLASS')
with col[1]:
    st.subheader('Molecular LogP')
    st.metric('Large', mollogp_large)
    st.metric('Small', mollogp_small)
    st.bar_chart(df_class, x='MOLWT_CLASS', y='MOLLOGP', color='MOLWT_CLASS')

    st.subheader('Aromatic Proportion')
    st.metric('Large', mollogp_large)
    st.metric('Small', mollogp_small)
    st.bar_chart(df_class, x='MOLWT_CLASS', y='AROMATICPROPORTION', color='MOLWT_CLASS')

with st.expander('Show Original DataFrame'):
    st.dataframe(df)
with st.expander('Show Aggregated DataFrame'):
    st.dataframe(df_class)

## References

- [ESOL:  Estimating Aqueous Solubility Directly from Molecular Structure](https://pubs.acs.org/doi/10.1021/ci034243x)
- [st.bar_chart](https://docs.streamlit.io/develop/api-reference/charts/st.bar_chart)
- [st.expander](https://docs.streamlit.io/develop/api-reference/layout/st.expander)
- [st.slider](https://docs.streamlit.io/develop/api-reference/widgets/st.slider)